In [5]:
#Many of these prep functions have been found around the web and/or borrowed from https://nbviewer.jupyter.org/url/norvig.com/ipython/Advent%20of%20Code.ipynb

# Python 3.x
import re
import numpy as np
import math
import urllib.request

from collections import Counter, defaultdict, namedtuple, deque
from functools   import lru_cache
from itertools   import permutations, combinations, chain, cycle, product, islice
from heapq       import heappop, heappush

def Input(day):
    "Open this day's input file."
    filename = '/home/isaac/AdventOfCode2018/day{}.txt'.format(day)
    return open(filename)
    
def input_lines(day):
    fobj = Input(day)
    lines = fobj.readlines()
    fobj.close()
    return lines

def transpose(matrix): return zip(*matrix)

def first(iterable): return next(iter(iterable))

def nth(iterable, n, default=None):
    "Returns the nth item of iterable, or a default value"
    return next(islice(iterable, n, None), default)

cat = ''.join

Ø   = frozenset() # Empty set
inf = float('inf')
BIG = 10 ** 999

def grep(pattern, lines):
    "Print lines that match pattern."
    for line in lines:
        if re.search(pattern, line):
            print(line)
            
def gengrep(pattern, lines):
    patc = re.compile(pattern)
    return (line for line in lines if patc.search(line))

def groupby(iterable, key=lambda it: it):
    "Return a dic whose keys are key(it) and whose values are all the elements of iterable with that key."
    dic = defaultdict(list)
    for it in iterable:
        dic[key(it)].append(it)
    return dic

def powerset(iterable):
    "Yield all subsets of items."
    items = list(iterable)
    for r in range(len(items)+1):
        for c in combinations(items, r):
            yield c

# 2-D points implemented using (x, y) tuples
def X(point): return point[0]
def Y(point): return point[1]

def neighbors4(point): 
    "The four neighbors (without diagonals)."
    x, y = point
    return ((x+1, y), (x-1, y), (x, y+1), (x, y-1))

def neighbors8(point): 
    "The eight neighbors (with diagonals)."
    x, y = point 
    return ((x+1, y), (x-1, y), (x, y+1), (x, y-1),
            (X+1, y+1), (x-1, y-1), (x+1, y-1), (x-1, y+1))

def cityblock_distance(p, q=(0, 0)): 
    "City block distance between two points."
    return abs(X(p) - X(q)) + abs(Y(p) - Y(q))

def euclidean_distance(p, q=(0, 0)): 
    "Euclidean (hypotenuse) distance between two points."
    return math.hypot(X(p) - X(q), Y(p) - Y(q))

def trace1(f):
    "Print a trace of the input and output of a function on one line."
    def traced_f(*args):
        result = f(*args)
        print('{}({}) = {}'.format(f.__name__, ', '.join(map(str, args)), result))
        return result
    return traced_f

def astar_search(start, h_func, moves_func):
    "Find a shortest sequence of states from start to a goal state (a state s with h_func(s) == 0)."
    frontier  = [(h_func(start), start)] # A priority queue, ordered by path length, f = g + h
    previous  = {start: None}  # start state has no previous state; other states will
    path_cost = {start: 0}     # The cost of the best path to a state.
    while frontier:
        (f, s) = heappop(frontier)
        if h_func(s) == 0:
            return Path(previous, s)
        for s2 in moves_func(s):
            new_cost = path_cost[s] + 1
            if s2 not in path_cost or new_cost < path_cost[s2]:
                heappush(frontier, (new_cost + h_func(s2), s2))
                path_cost[s2] = new_cost
                previous[s2] = s
    return dict(fail=True, front=len(frontier), prev=len(previous))
                
def Path(previous, s): 
    "Return a list of states that lead to state s, according to the previous dict."
    return ([] if (s is None) else Path(previous, previous[s]) + [s])

In [6]:
#day1 part 1 
#Simply sum all the lines of the input
sum((int(l.strip()) for l in input_lines(1)))

425

In [30]:
#day1 part2
#Do the same as in part 1, but keep a running list of intermediate values
#find the first value to be reached twice
input_generator = cycle(int(l.strip()) for l in input_lines(1))
visited = [0]
for iv in input_generator:
    nxt = visited[-1]+iv
    if nxt in visited:
        break
    visited.append(nxt)
print(nxt)
print(visited[-1])

57538
57521


In [39]:
#day2 part1
#Count the number of strings that have any letter appearing exactly three times
#Count the number of strings that have any letter appearing exactly two times
#multiply those numbers to get answer
lines = input_lines(2)
g1 = (Counter(l) for l in lines)
g2 = (Counter(l) for l in lines)
threes = sum(1 for x in g1 if 3 in x.values())
twos = sum(1 for x in g2 if 2 in x.values())
threes*twos

5928

In [55]:
#day2 part2
#find the ids that differ by only one letter in the same position
lines = input_lines(2)
def nletterdiff(s1, s2):
    return sum(1 for l1,l2 in zip(s1,s2) if l1!=l2)
def find_nearest_match(lns):
    for c in combinations(lns, 2):
        if nletterdiff(*c)==1:
            return c
def remove_differing_letter(s1,s2):
     return cat(l1 for l1,l2 in zip(s1,s2) if l1==l2)
print(remove_differing_letter(*find_nearest_match(lines)))
            

bqlporuexkwzyabnmgjqctvfs



In [58]:
#day3 part1
#find number of square inches in rectangles that are claimed by more than one sub-rectangle
lines = input_lines(3)
def process_line(line):
    id, _, a, b = line.strip().split()
    min_x, min_y = map(int, a.strip(':').split(','))
    sz_x, sz_y = map(int, b.split('x'))
    return id, min_x, min_y, min_x+sz_x, min_y+sz_y
def find_max_dimensions(lns):
    maxx = 0
    maxy = 0
    for line in lns:
        *_, max_x, max_y = process_line(line)
        maxx = max_x if max_x>maxx else maxx
        maxy = max_y if max_y>maxy else maxy
    return maxx, maxy
def claim_matrix(lns):
    fabric = np.zeros(find_max_dimensions(lns), dtype='int')
    for id, min_x, min_y, max_x, max_y in map(process_line, lns):
        fabric[min_x:max_x, min_y:max_y]+=1
    return fabric
def n_overlap(lns):
    mtx = claim_matrix(lns)
    return np.sum(mtx>1)
n_overlap(lines)

115242

In [60]:
#day3 part2
#Find the claim that does not overlap any others
#Can do only two iterations by building the claim matrix, then checking each claim against the matrix
lines = input_lines(3)
def is_unique(sub_matrix):
    return np.product(sub_matrix.shape) == np.sum(sub_matrix)
def find_unique_claim(lns):
    mtx = claim_matrix(lns)
    for id, min_x, min_y, max_x, max_y in map(process_line, lns):
        if is_unique(mtx[min_x:max_x, min_y:max_y]):
            return id
find_unique_claim(lines)

'#1046'

In [69]:
#day4 part1
#Sort Entries
#Find guard with most minutes slept, and minute with most sleep.
lines4 = input_lines(4)
import datetime
def process_lines(lns):
    fmtstring = '%Y-%m-%d %H:%M'
    return sorted([(datetime.datetime.strptime(l.split(']')[0].strip('['), fmtstring), 
                    l.split(']')[1].strip().rstrip()) for l in lns],
                  key=lambda x: x[0])
process_lines(lines4)

def guard_records(lns):
    sorted_lines = process_lines(lns)
    guards = defaultdict(list)
    current_guard = 0
    for time, event in sorted_lines:
        if 'begins shift' in event:
            current_guard = int(event.split()[1].strip('#'))
        guards[current_guard].append((time, event))
    return guards
            
def minutes_asleep_counter(guard_rec):
    counter = np.zeros(60)
    awake = True
    asleep=0
    for time, event in guard_rec:
        if 'begins shift' in event:
            awake=True
            asleep = 0
        elif event=='falls asleep':
            awake=False
            asleep = time.minute
        elif event == 'wakes up':
            awake=True
            counter[asleep:time.minute] += 1
    return counter

def guard_counters(lns):
    records = guard_records(lns)
    #grds = {}
    max_asleep = 0
    max_minute = 0
    max_guard = 0
    for guard, record in records.items():
        ctr = minutes_asleep_counter(record)
        asleep = np.sum(ctr)
        if asleep > max_asleep:
            max_asleep = asleep
            max_minute = np.argmax(ctr)
            max_guard = guard
        #grds[guard] = {'asleep':asleep, 'minute':np.argmax(ctr)}
    return max_guard, max_asleep, max_minute, max_guard*max_minute
guard_counters(lines4)

(641, 498.0, 41, 26281)

In [87]:
#day4 part2
def guard_counters2(lns):
    records = guard_records(lns)
    grds = {}
    for guard, record in records.items():
        grds[guard] = minutes_asleep_counter(record)
    return grds

def strategy2(counter_dict):
    guards, minute_counters = list(zip(*counter_dict.items()))
    minute_matrix = np.array(minute_counters)
    
    #return guards, minute_matrix.shape, np.max(minute_matrix, axis=1), np.argwhere(minute_matrix>=np.max(minute_matrix))
    gd, min = tuple(np.argwhere(minute_matrix>=np.max(minute_matrix))[0])
    return guards[gd]*min
ctrs = guard_counters2(lines4)
print(strategy2(ctrs))

73001


In [119]:
#day5 part1:
#process polymer(string)
#rules:
#If two adjacent letters are equal with opposite polarization, they annhilate
#iterate with this rule until no more pairs of letters with equal and opposite polarization exist

def find_all_pairs(strng):
    pair_locs = []
    last_ch = ''
    for i, ch in enumerate(strng):
        if ch.upper()==last_ch.upper() and last_ch.isupper()+ch.isupper()==1:
            if len(pair_locs)==0 or i != pair_locs[-1]+1:
                pair_locs.append(i)
        last_ch = ch
    return pair_locs

def remove_pairs(strng, pair_list):
    if len(pair_list)==0:
        return strng
    lst = [strng[:pair_list[0]-1]]
    last_index = pair_list[0]
    try:
        for index in pair_list[1:]:
            lst.append(strng[last_index+1:index-1])
            last_index=index
    except IndexError:
        pass
    lst.append(strng[last_index+1:])
    return cat(lst)

def react_all(strng):
    last_string = strng
    newstring = ''
    while True:
        pairs = find_all_pairs(last_string)
        newstring = remove_pairs(last_string, pairs)
        if newstring==last_string:
            return len(newstring)
        last_string=newstring
        
line5 = input_lines(5)[0].strip()
react_all(line5)

11540

In [120]:
#day5 part 2
#try 26 different versions.  In each, remove all instances of upper and lower versions of a single letter,
#then react the new string
#find the length of the shortest new string
import string

def remove_all(strng, unit):
    return strng.replace(unit.lower(),'').replace(unit.upper(),'')

def find_shortest_fixed(strng):
    min_len = len(strng)
    for unit in string.ascii_lowercase:
        test_strng = remove_all(strng, unit)
        newl = react_all(test_strng)
        min_len = newl if newl < min_len else min_len
    return min_len

find_shortest_fixed(line5)

6918